In [1]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from nltk.corpus import stopwords
import pickle 
import math

In [2]:
mask = pipeline('fill-mask')

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
sentiment_model = pipeline('sentiment-analysis')

In [3]:
def idx2str(s):
    return ' '.join([id2w[idx] for idx in s])

def str2idx(idx):
    idx = idx.strip()
    return [w2id[idxx] for idxx in idx.split()]

In [4]:
Y_train, C_train = pickle.load(open('AEGS_data/amazon/XC_train.pkl','rb'))
Y_dev, C_dev = pickle.load(open('AEGS_data/amazon/XC_dev.pkl','rb'))
Y_test, C_test = pickle.load(open('AEGS_data/amazon/XC_test.pkl','rb'))

with open('AEGS_data/amazon/w2id_id2w.pkl', 'rb') as f:
    w2id, id2w = pickle.load(f)

In [5]:
original, reference, original_noun, reference_noun = pickle.load(open('AEGS_data/amazon/original_reference_and_noun.pkl','rb'))
C_original = [[1, 0] for i in range(500)] + [[0, 1] for i in range(500)]

X_train = [x[:-1] for x in Y_train]
X_dev = [x[:-1] for x in Y_dev]
X_test = [x[:-1] for x in Y_test]
print(idx2str(Y_test[1]), idx2str(X_test[1]))

they are thinner and retain memory than the older ones . <EOS> they are thinner and retain memory than the older ones .


In [12]:
i=1
idx2str(original[i]), idx2str(reference[i]), original_noun[i], reference_noun[i]

('shame on what otherwise has been a reliable company with several other <UNK> products .',
 'these fit so well with what has been a reliable company with several other <UNK> products .',
 [('shame', 'NN'),
  ('otherwise', 'NN'),
  ('company', 'NN'),
  ('products', 'NNS')],
 [('company', 'NN'), ('products', 'NNS')])

In [13]:
# sentence = "I am going to guess everything in this sentence"
sentence = idx2str(reference[i]).replace(' <UNK> ', ' ').replace('.', '').replace(' i ', '  I ')
sentence_split = sentence.split()
stop_words = set(stopwords.words('english'))
keywords = [w for w in sentence_split if not w in stop_words]

In [14]:
alpha = [0.001, 0.003, 0.005, 0.01, 0.03, 0.05, 0.1]
final_keys = {}
for a in alpha:
    final_keys[a] = []
orig_sent = sentiment_model(sentence)[0]
orig_score = 0 
if orig_sent['label'] == "NEGATIVE":
    orig_score -= orig_sent['score']
else:
    orig_score += orig_sent['score']
    
for key in keywords:
    s = sentence.replace(key, '<mask>')
    candidates = mask(s, top_k=20)
#     print(candidates[15]['sequence'], candidates[17]['sequence'], candidates[19]['sequence'])
    cand1 = sentiment_model(candidates[15]['sequence'])
    cand2 = sentiment_model(candidates[17]['sequence'])
    cand3 = sentiment_model(candidates[19]['sequence'])
    cand_score = 0
    for cand in [cand1, cand2, cand3]:
        print(cand)
        if cand[0]['label'] == 'NEGATIVE':
            cand_score -= cand[0]['score']
        else:
            cand_score += cand[0]['score']
    
    cand_score /= 3
    print(orig_score, cand_score, key)
    for a in alpha:
        if abs(orig_score - cand_score) <= a:
            final_keys[a].append(key)  

[{'label': 'POSITIVE', 'score': 0.9994624853134155}]
[{'label': 'POSITIVE', 'score': 0.9989681243896484}]
[{'label': 'POSITIVE', 'score': 0.9990249276161194}]
0.9991846084594727 0.9991518457730612 fit
[{'label': 'POSITIVE', 'score': 0.9987897276878357}]
[{'label': 'POSITIVE', 'score': 0.9897065162658691}]
[{'label': 'POSITIVE', 'score': 0.9886536002159119}]
0.9991846084594727 0.9923832813898722 well
[{'label': 'POSITIVE', 'score': 0.9998316764831543}]
[{'label': 'POSITIVE', 'score': 0.9886431097984314}]
[{'label': 'POSITIVE', 'score': 0.9992959499359131}]
0.9991846084594727 0.9959235787391663 reliable
[{'label': 'POSITIVE', 'score': 0.9971632361412048}]
[{'label': 'POSITIVE', 'score': 0.9975175857543945}]
[{'label': 'POSITIVE', 'score': 0.9989576935768127}]
0.9991846084594727 0.9978795051574707 company
[{'label': 'POSITIVE', 'score': 0.9988303780555725}]
[{'label': 'POSITIVE', 'score': 0.9979692697525024}]
[{'label': 'POSITIVE', 'score': 0.9976409077644348}]
0.9991846084594727 0.998146

In [15]:
final_keys

{0.001: ['fit', 'products'],
 0.003: ['fit', 'company', 'several', 'products'],
 0.005: ['fit', 'reliable', 'company', 'several', 'products'],
 0.01: ['fit', 'well', 'reliable', 'company', 'several', 'products'],
 0.03: ['fit', 'well', 'reliable', 'company', 'several', 'products'],
 0.05: ['fit', 'well', 'reliable', 'company', 'several', 'products'],
 0.1: ['fit', 'well', 'reliable', 'company', 'several', 'products']}

In [16]:
keywords 

['fit', 'well', 'reliable', 'company', 'several', 'products']